In [45]:
import requests
import json
import time
import pandas as pd
import re
from datetime import datetime

token = ''
domain = 'bodyfitsecret'

wall_inf_0 = requests.get('https://api.vk.com/method/wall.get', params = {'access_token' : token,
                                                                          'v' : 5.126,
                                                                          'domain' : domain,
                                                                          'count' : 1
                                                                         }
                         )

posts_count = int(wall_inf_0.json()['response']['count'])
owner_id = wall_inf_0.json()['response']['items'][0]['owner_id']
print(posts_count)
print(owner_id)

48836
-61040032


In [46]:
offset = 0

all_posts_inf = []

while offset <= posts_count * 0.001: # Извлекать конкрентную долю записей необязательно
    wall_inf_0 = requests.get('https://api.vk.com/method/wall.get', params = {'access_token' : token,
                                                                              'v' : 5.126,
                                                                              'domain' : domain,
                                                                              'count' : 100,
                                                                              'offset' : offset
                                                                             }
                             )
    offset += 100
    posts_inf = wall_inf_0.json()['response']['items']
    all_posts_inf.extend(posts_inf)

In [47]:

id_list = []
date_list = []
text_list = []
comments_count_list = []
likes_count_list = []
reposts_count_list = []
views_count_list = []

for post in all_posts_inf:
    keys_list = list(post.keys())
    if 'views' in keys_list and 'likes' in keys_list and 'comments' in keys_list and 'reposts' in keys_list:
        for k,v in post.items():
            if k == 'id':
                id_list.append(v)
            if k == 'date':
                date_list.append(datetime.utcfromtimestamp(v).strftime('%d-%m-%Y'))
            if k == 'text':
                if v == '':
                    text_list.append('-')
                else:
                    text_list.append(v)
            if k == 'comments':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        comments_count_list.append(v1)
            if k == 'likes':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        likes_count_list.append(v1)
            if k == 'reposts':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        reposts_count_list.append(v1)
            if k == 'views':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        views_count_list.append(v1)

In [48]:
id_list = []
date_list = []
text_list = []
comments_count_list = []
likes_count_list = []
reposts_count_list = []
views_count_list = []

for post in all_posts_inf:
    keys_list = list(post.keys())
    if 'views' in keys_list and 'likes' in keys_list and 'comments' in keys_list and 'reposts' in keys_list:
        for k,v in post.items():
            if k == 'id':
                id_list.append(v)
            if k == 'date':
                date_list.append(datetime.utcfromtimestamp(v).strftime('%d-%m-%Y'))
            if k == 'text':
                if v == '':
                    text_list.append('-')
                else:
                    text_list.append(v)
            if k == 'comments':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        comments_count_list.append(v1)
            if k == 'likes':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        likes_count_list.append(v1)
            if k == 'reposts':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        reposts_count_list.append(v1)
            if k == 'views':
                for k1,v1 in v.items():
                    if k1 == 'count':
                        views_count_list.append(v1)

In [49]:
comments_list = []

comments_count_num = 0

for post_id in id_list:
    comment_inf_0 = requests.get('https://api.vk.com/method/wall.getComments', params = {'access_token' : token,
                                                                                         'v' : 5.126,
                                                                                         'owner_id' : owner_id,
                                                                                         'post_id' : post_id,
                                                                                         'count' : comments_count_list[comments_count_num]
                                                                                        }
                                )
    comment_inf_1 = comment_inf_0.json()['response']['items']
    comments_post_list = []
    for comment in comment_inf_1:
        for k,v in comment.items():
            if k == 'text':
                if v == '':
                    comments_post_list.append('-')
                else:
                    comments_post_list.append(v)
    comments_list.append(comments_post_list)
    comments_count_num += 1
    time.sleep(0.5)

In [50]:
col_names = ['ID',
             'Дата',
             'Текст записи',
             'Комментарии к записи',
             'Число просмотров',
             'Число лайков',
             'Число репостов',
             'Число комментариев'
            ]

posts_inf_df = pd.DataFrame([id_list,
                             date_list,
                             text_list,
                             comments_list,
                             views_count_list,
                             likes_count_list,
                             reposts_count_list,
                             comments_count_list
                            ]
                           )

posts_inf_df = posts_inf_df.T

posts_inf_df.columns = col_names

posts_inf_df

,ID,Дата,Текст записи,Комментарии к записи,Число просмотров,Число лайков,Число репостов,Число комментариев
0,224180,02-05-2021,Отличная йога для вашего животика! Будь в форме!,[Не одно из этих упражнений не поможет : !если...,79495,1682,1676,14
1,229122,20-07-2021,Убираем жир между ног👌🏻,"[Жир... Между ног...??? У меня всё.🤣🤣🤣🤣☺☺☺, От...",2303,18,26,2
2,229119,20-07-2021,Для шикарных ягодиц😍,[],9978,82,88,0
3,229117,20-07-2021,Подтягиваем ягодицы🔥,"[Судя по ягодицам девушки ,упражнение не эффе...",14515,136,119,1
4,229106,20-07-2021,Убираем низ живота 👌🏻,[],15454,167,171,0
...,...,...,...,...,...,...,...,...
95,228370,07-07-2021,Это дебетовая карта Tinkoff Black\nЗа покупки ...,[],11425,3,2,0
96,228366,07-07-2021,‼Работает только такая формула:👇⁣ \n✅ Фитнес ＋...,[],47419,706,678,0
97,228357,07-07-2021,"Упражнения, которые помогут вам поднять ягодиц...",[],62600,663,632,0
98,228355,07-07-2021,Укрепляем мышцы груди,[В груди мышц нет могли бы знать],63950,715,728,3


In [51]:
membors_inf_0 = requests.get('https://api.vk.com/method/groups.getMembers', params = {'access_token' : token,
                                                                                      'v' : 5.126,
                                                                                      'group_id' : -owner_id,
                                                                                      'count' : 1
                                                                                     }
                            )

membors_count = int(membors_inf_0.json()['response']['count'])

offset = 0

membors = []

while offset < membors_count * 0.01:
    membors_inf_0 = requests.get('https://api.vk.com/method/groups.getMembers', params = {'access_token' : token,
                                                                                          'v' : 5.126,
                                                                                          'group_id' : -owner_id,
                                                                                          'count' : 1000,
                                                                                          'sort' : 'id_desc',
                                                                                          'fields' : ['sex, bdate, city, education']
                                                                                         }
                                )
    membors_inf_1 = membors_inf_0.json()['response']['items']
    membors.extend(membors_inf_1)
    offset += 1000
    
print (len(membors))

11000


In [52]:
members_id = []
members_first_name = []
members_last_name = []
members_sex = []
members_bdate = []
members_city = []
members_university_fact = []

members_id_x = 0
members_first_name_x = 0
members_last_name_x = 0
members_sex_x = 0
members_bdate_x = 0
members_city_x = 0
members_university_fact_x = 0

for member in membors:    
    for k,v in member.items():
        if 'id' in member:
            if k == 'id':
                members_id.append(v)
        else:
            if members_id_x == 0:
                members_id.append('-')
                members_id_x = 1
        if 'first_name' in member:
            if k == 'first_name':
                members_first_name.append(v)
        else:
            if members_first_name_x == 0:
                members_first_name.append('-')
                members_first_name_x = 1
        if 'last_name' in member:
            if k == 'last_name':
                members_last_name.append(v)
        else:
            if members_last_name_x == 0:
                members_last_name.append('-')
                members_last_name_x = 1
        if 'sex' in member:
            if k == 'sex':
                if v == 1:
                    members_sex.append('Женщина')
                else:
                    members_sex.append('Мужчина')
        else:
            if members_sex_x == 0:
                members_sex.append('-')
                members_sex_x = 1
        if 'bdate' in member:
            if k == 'bdate':
                b_year = re.search(r'\d\d\d\d', v)
                if b_year == None:
                    members_bdate.append('-')
                else:
                    members_bdate.append(int(b_year.group(0)))
        else:
            if members_bdate_x == 0:
                members_bdate.append('-')
                members_bdate_x = 1
        if 'city' in member:
            if k == 'city':
                for k1,v1 in v.items():
                    if k1 == 'title':
                        members_city.append(v1)
        else:
            if members_city_x == 0:
                members_city.append('-')
                members_city_x = 1
        if 'university' in member:
            if k == 'university':
                members_university_fact.append('Образован')
        else:
            if members_university_fact_x == 0:
                members_university_fact.append('-')
                members_university_fact_x = 1
    
    members_id_x = 0
    members_first_name_x = 0
    members_last_name_x = 0
    members_sex_x = 0
    members_bdate_x = 0
    members_city_x = 0
    members_university_fact_x = 0

In [53]:
col_names = ['ID пользователя',
             'Имя пользователя',
             'Фамилия пользователя',
             'Пол пользователя',
             'Год рождения пользователя',
             'Город проживания пользователя',
             'Наличие образования у пользователя'
            ]

membors_inf_df = pd.DataFrame([members_id,
                               members_first_name,
                               members_last_name,
                               members_sex,
                               members_bdate,
                               members_city,
                               members_university_fact
                              ]
                             )

membors_inf_df = membors_inf_df.T

membors_inf_df.columns = col_names

membors_inf_df

,ID пользователя,Имя пользователя,Фамилия пользователя,Пол пользователя,Год рождения пользователя,Город проживания пользователя,Наличие образования у пользователя
0,664773690,Liya,Bazhenova,Женщина,-,-,-
1,664704800,Lora,Rubinkina,Женщина,1996,-,Образован
2,664634151,Karina,Koroleva,Женщина,1991,-,Образован
3,664603016,Tatyana,Tigaeva,Женщина,1993,-,Образован
4,664594097,Anna,Semkova,Женщина,-,-,-
...,...,...,...,...,...,...,...
10995,651299551,Lidiya,Inouye,Женщина,-,-,-
10996,651299421,Lyubov,Monti,Женщина,-,-,-
10997,651283575,Alexandra,Bautkina,Женщина,2007,-,Образован
10998,651263908,Naninedrig,Dandanyan,Мужчина,2002,-,-


In [54]:
!pip install xlsxwriter
import xlsxwriter
path = '/content/bodyfitsecret.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
posts_inf_df.to_excel(writer, sheet_name = 'posts')
membors_inf_df.to_excel(writer, sheet_name = 'membors')
writer.save()
writer.close()

/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
